In [100]:
import nltk
nltk.download('state_union')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
import re
from nltk.parse.stanford import StanfordDependencyParser

[nltk_data] Downloading package state_union to
[nltk_data]     /home/elias/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package punkt to /home/elias/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/elias/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [101]:
dependancy_parser = StanfordDependencyParser(
    path_to_jar = 'stanford-parser-full-2018-10-17/stanford-parser.jar',
    path_to_models_jar = 'stanford-english-corenlp-2018-02-27-models.jar')

/home/elias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: 'list_iterator' object has no attribute 'nodes'

In [103]:
dependancy_parser = StanfordDependencyParser(
    path_to_jar = 'stanford-parser-full-2018-10-17/stanford-parser.jar',
    path_to_models_jar = 'stanford-english-corenlp-2018-02-27-models.jar')


/home/elias/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The StanfordDependencyParser will be deprecated
Please use nltk.parse.corenlp.CoreNLPDependencyParser instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [104]:
def search_for_dep_with_pos(tree, deps, collection, pos_list):
    for pos in pos_list:
        if pos in deps.keys():
            for j in deps[pos]:
                collection.append((tree.nodes[j]['word'], pos))
    return collection

def clean_search_results(search_results):
    new_results = []
    for guess in search_results:
        valid = True
        for pos in guess.keys():
            if guess[pos] == []:
                valid = False
        if valid:
            new_results.append(guess)
    return new_results



In [105]:
s = 'Pittsburgh is in Pennsylvania'
result = dependancy_parser.raw_parse(s)
for node in result: print(node)

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x7efcd3168d90>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': [4]}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'NNP',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 4,
                 'lemma': '_',
                 'rel': 'nsubj',
                 'tag': 'NNP',
                 'word': 'Pittsburgh'},
             2: {'address': 2,
                 'ctag': 'VBZ',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 4,
                 'lemma': '_',
                 'rel': 'cop',
                 'tag': 'VBZ',
       

In [106]:
def get_verb_subj_dobj(tree):
    results = []
    for i in range(len(tree.nodes)):
        if tree.nodes[i]['tag'][:2] == 'VB':
            subject = []
            dir_object = []
            verb = []
            verb.append(tree.nodes[i]['word'])
            deps = tree.nodes[i]['deps']
            subject = search_for_dep_with_pos(tree, deps, subject, ['nsubj', 'nsubjpass'])
            dir_object = search_for_dep_with_pos(tree, deps, dir_object, ['nmod', 'dobj'])
            verb = search_for_dep_with_pos(tree, deps, verb, ['auxpass'])
            results.append({"Subject": subject, "Direct Object": dir_object, "Verb": verb})
    return clean_search_results(results)

def get_noun_descriptor(tree):
    results = []
    for i in range(len(tree.nodes)):
        subject = []
        descriptor = []
        verb = []
        case = [('', 'ph')]
        deps = tree.nodes[i]['deps']
        descriptor.append((tree.nodes[i]['word']))
        subject = search_for_dep_with_pos(tree, deps, subject, ['nsubj'])
        verb = search_for_dep_with_pos(tree, deps, verb, ['cop'])
        case = search_for_dep_with_pos(tree, deps, case, ['case'])
        results.append({"Subject": subject, "Descriptor": descriptor, "Verb": verb, "Case": case})
    return clean_search_results(results)
                   
s = 'Pittsburgh is in Pennsylvania'
result = dependancy_parser.raw_parse(s)
for node in result: print(get_noun_descriptor(node))


[{'Subject': [('Pittsburgh', 'nsubj')], 'Descriptor': ['Pennsylvania'], 'Verb': [('is', 'cop')], 'Case': [('', 'ph'), ('in', 'case')]}]


q_templates = [
"When did {} {}?",
"Where did {} {}?",
"What did {} do?",
"What did {} do in {}?"
]

In [9]:
q_templates = [ 
    "When did {} {}?", 
    "Where did {} {}?", 
    "What did {} do?", 
    "What did {} do in {}?", 
    "Who {}?"
]

class question_template:
    
    def __init__(self, string):
        self.string = string
        

In [10]:
def generate_questions(sentence):
    parse_tree = dependancy_parser.raw_parse(sentence)
    dep = next(parse_tree)
    simplified = list(dep.triples())
    get_subjects(simplified, sentence)
    get_objects(simplified, sentence)
    print(simplified)
    
def get_subjects(pt, sentence):
    all_subjects = []
    for entry in pt:
        if entry[1] == 'nsubj' or entry[1] == 'nmod':
            subject = [entry[2][0]]
            for entry_pass_2 in pt:
                if entry_pass_2[0][0] == entry[2][0] and entry_pass_2[1] == 'compound':
                    subject.append(entry_pass_2[2][0])
            return_subject = ''

            for word in re.sub(',','',sentence).split():
                if word in subject:
                    return_subject += word
                    return_subject += ' '
            if entry[0][1][:2] == 'VB':
                print('What did {} {}?'.format(return_subject, entry[0][0]))
            all_subjects.append(return_subject)
    return all_subjects

def get_objects(pt, sentence):
    all_objects = []
    for entry in pt:
        if entry[1] == 'dobj' or entry[1] == 'nsubjpass':
            object_ = [entry[2][0]]
            for entry_pass_2 in pt:
                if entry_pass_2[0][0] == entry[2][0] and entry_pass_2[1] == 'compound':
                    object_.append(entry_pass_2[2][0])
            return_object = ''

            for word in re.sub(',','',sentence).split():
                if word in object_:
                    return_object += word
                    return_object += ' '
            if entry[0][1][:2] == 'VB':
                print('Who {} {}?'.format(entry[0][0], return_object))
            all_objects.append(return_object)
    return all_objects

                    



NameError: name 'dependancy_parser' is not defined

'as'

[1, 2, 3]

[True, True, True, True, True]


True

FizzBuzz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz
Fizz
22
23
Fizz
Buzz
26
Fizz
28
29
FizzBuzz
31
32
Fizz
34
Buzz
Fizz
37
38
Fizz
Buzz
41
Fizz
43
44
FizzBuzz
46
47
Fizz
49
Buzz
Fizz
52
53
Fizz
Buzz
56
Fizz
58
59
FizzBuzz
61
62
Fizz
64
Buzz
Fizz
67
68
Fizz
Buzz
71
Fizz
73
74
FizzBuzz
76
77
Fizz
79
Buzz
Fizz
82
83
Fizz
Buzz
86
Fizz
88
89
FizzBuzz
91
92
Fizz
94
Buzz
Fizz
97
98
Fizz
Buzz


In [200]:
(3 and 5)

5